In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

Restart Session before running next cell

In [1]:
!pip install sacrebleu

In [16]:
print("Select the language:")
print("1. Hindi")
print("2. Kannada")
print("3. Gujarati")

# Get user input for language selection
selected_option = input("Enter the number corresponding to the desired language: ")

# Map user input to language code
language_mapping = {
    '1': 'hi',  # Hindi
    '2': 'kn',  # Kannada2
    '3': 'gu',  # Gujarati
}

# Validate user input and get the corresponding language code
selected_language = language_mapping.get(selected_option)

Select the language:
1. Hindi
2. Kannada
3. Gujarati


Enter the number corresponding to the desired language:  3


In [17]:
input_paragraph = input(f"Enter a paragraph in {selected_language} script:\n")


Enter a paragraph in gu script:
 પોલ બ્રન્ટન ઇગ્લેન્ડના એક છાપાંના તંત્રી, પત્રકાર. ભારતના પ્રાચીન ગૌરવ અને એની સંતપરંપરાથી આકર્ષાયેલા પોલ બ્રન્ટન અનેકવાર ભારતનો પ્રવાસ કરવા આવેલા. એ દરમિયાન એમણે ભારતમાં પ્રવર્તતા અનેકવિધ મત-સંપ્રદાયના સંતોમહંતો, યોગીફકીરો સાથે રૂબરૂ મુલાકાતો કરી અને એના આધારે કેટલાંક પુસ્તકો લખ્યાં. એ શ્રેણીનું પહેલું પુસ્તક એટલે


In [18]:
hi_sents = input_paragraph.split('\n')

In [19]:
if selected_language == 'hi' : 
    src_lang, tgt_lang = "hin_Deva", "eng_Latn"
elif selected_language == 'kn' :
    src_lang, tgt_lang = "kan_Knda", "eng_Latn"
elif selected_language == 'gu' : 
    src_lang, tgt_lang = "guj_Gujr", "eng_Latn"
    


In [20]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [21]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [22]:
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", "")
ip = IndicProcessor(inference=True)

/tmp/ipykernel_180/2021851695.py:17: DeprecationWarning: This IndicTransTokenizer is deprecated.
The official Tokenizer is available on HF and can be used as follows:
```
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
```
  tokenizer = IndicTransTokenizer(direction=direction)


In [23]:


# hi_paragraph =  """ सचिन तेंदुलकर ने एकदिवसीय अंतरराष्ट्रीय क्रिकेट में इतिहास रचने के बाद अपने दोहरे शतक को भारतीय लोगों को समर्पित करते हुए खुलासा किया कि जब वह 175 रन के पार पहुंच गये थे तब उन्हें लगा कि 200 रन के जादुई आंकड़े को छुआ जा सकता है.
# तेंदुलकर ने दक्षिण अफ्रीका के खिलाफ दूसरे एकदिवसीय मैच में नाबाद 200 रन बनाये. वह वन डे में दोहरा शतक जड़ने वाले दुनिया के पहले बल्लेबाज बने जिससे भारत ने यह मैच 153 रन के विशाल अंतर से जीता. उन्हें इस पारी के लिये मैन आफ द मैच चुना गया. तेंदुलकर ने मैच के बाद भारतीय क्रिकेट प्रेमियों का आभार व्यक्त किया.
# उन्होंने कहा, ‘मैं नहीं जानता कि इस उपलब्धि पर कैसे प्रतिक्रिया करूं. मैं इस दोहरे शतक को भारतीय जनता को समर्पित करता हूं जो पिछले 20 साल से मेरे उतार चढ़ावों में हमेशा मेरा साथ देते रहे हैं. मैं उनके सहयोग को कभी नहीं भुला सकता.’ मास्टर ब्लास्टर ने इसके साथ ही खुलासा किया कि जब वह 175 रन के पार पहुंच गये थे तभी उन्होंने दोहरा शतक जड़ने के बारे में सोचा क्योंकि तब 42 ओवर ही हुए थे.
# उन्होंने कहा, ‘जब मैं 175 रन के पार पहुंच गया और 42वां ओवर चल रहा था तब मुझे लगा कि मैं दोहरे शतक तक पहुंच सकता हूं लेकिन मैं इसी के बारे में नहीं सोच रहा था. जब मैं इसके बेहद करीब पहुंच गया तभी इस बारे में मैंने सोचना शुरू किया.’ तेंदुलकर ने कहा कि वह अपनी क्रिकेट का पूरा लुत्फ उठा रहे हैं और उन्हें खुशी है कि उनकी पारी से टीम ने बड़ी जीत दर्ज की. उन्होंने कहा, ‘मैं अपने खेल का पूरा लुत्फ उठा रहा हूं और मैंने टीम के लिये जो सर्वश्रेष्ठ हो वही करने का प्रयास किया है.
# बल्लेबाज के तौर पर मैंने कुछ गलत फैसले भी किये लेकिन मैंने हमेशा टीम की भलाई के बारे में ही सोचा.’ तेंदुलकर ने कहा, ‘50 ओवर तक बल्लेबाजी करना मेरी फिटनेस की अच्छी परीक्षा थी. पूरी पारी के दौरान मेरे शरीर ने पूरा साथ दिया. गेंद अच्छी तरह से बल्ले पर आ रही थी और मैंने अच्छे स्ट्रोक जमाये.’ इस स्टार बल्लेबाज ने इसके साथ कप्तान महेंद्र सिंह धोनी, दिनेश कार्तिक और यूसुफ पठान की भी तारीफ की जिन्होंने दूसरे छोर से उनका पूरा साथ दिया.
# उन्होंने कहा, ‘मेरा मानना था कि इस पिच पर 340 से 350 रन का स्कोर अच्छा रहेगा.’ उन्होंने कहा, ‘दिनेश कार्तिक, यूसुफ और धोनी ने बेहतरीन बल्लेबाजी की. यदि आप साझेदारी देखोगे तो प्रत्येक में बड़े शाट खेले गये. यूसुफ ने लय बदली और धोनी ने अंत में शानदार बल्लेबाजी की. इसलिए मैंने एक रन लेकर धोनी को स्ट्राइक देने की कोशिश की क्योंकि वह तब अच्छे शाट जमा रहा था. """
# hi_paragraph = input("Enter a paragraph in Hindi (Devanagari script):\n")


# src_lang, tgt_lang = "hin_Deva", "eng_Latn"
# Take input from the user
# input_sentence = input("Enter a sentence in Hindi (Devanagari script): ")
en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)
en_paragraph = ' '.join(en_translations)

print(en_paragraph)
# Display the translations
print(f"\n{src_lang} - {tgt_lang}")


Paul Brunton, a print editor, journalist from England. Paul Brunton, fascinated by India's ancient glory and its saintly traditions, visited India several times. During this time, he met and wrote several books based on Yogifakirs, the saints of the various faiths prevalent in India. This is the first book in the series.

guj_Gujr - eng_Latn


In [24]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [25]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("nijpadariya/Pegasus_fine_tune")
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained("nijpadariya/Pegasus_fine_tune").to(device)


In [26]:
model_pegasus


PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

In [27]:
#Tokenize the input text
inputs = tokenizer(en_paragraph, return_tensors="pt", max_length=1024, truncation=True)
inputs = {key: value.to(model_pegasus.device) for key, value in inputs.items()}
# Generate summary
summary_ids = model_pegasus.generate(inputs["input_ids"], max_length=150, num_beams=8, length_penalty=0.0, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Input Text:", en_paragraph)
print(" ")
print("Summary:", summary)

{'input_ids': tensor([[ 2040, 27488,  2035,   108,   114,  1591,  3704,   108,  8464,   135,
          2159,   107,  2040, 27488,  2035,   108, 14418,   141,  1144,   131,
           116,  3266,  7082,   111,   203, 19329,   445,  6760,   108,  3333,
          1144,   500,   488,   107,  2348,   136,   166,   108,   178,  1676,
           111,  2021,   500,  1031,   451,   124, 37735, 10368, 35332,   116,
           108,   109, 25000,   113,   109,   623,  2433,   116, 12273,   115,
          1144,   107,   182,   117,   109,   211,   410,   115,   109,   679,
           107,     1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}
tensor([[    0,  2040, 27488,  2035,   117, 14418,   141,  1144,   131,   116,
          3266,  7082, 

In [28]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)



/tmp/ipykernel_180/2021851695.py:17: DeprecationWarning: This IndicTransTokenizer is deprecated.
The official Tokenizer is available on HF and can be used as follows:
```
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
```
  tokenizer = IndicTransTokenizer(direction=direction)


In [29]:
en_sents = summary.split('\n')

if selected_language == 'hi' : 
    src_lang, tgt_lang = "eng_Latn","hin_Deva"
elif selected_language == 'kn' :
    src_lang, tgt_lang =  "eng_Latn","kan_Knda"
elif selected_language == 'gu' : 
    src_lang, tgt_lang = "eng_Latn","guj_Gujr"
    


hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

en_paragraph = ' '.join(hi_translations)


print(en_paragraph)
# Display the translations
print(f"\n{src_lang} - {tgt_lang}")

પોલ બ્રુનટન ભારતના પ્રાચીન ગૌરવ અને તેની સંત પરંપરાઓથી પ્રભાવિત છે. તેમણે ભારતમાં પ્રચલિત વિવિધ ધર્મોના સંતો યોગીફાકિર પર આધારિત અનેક પુસ્તકો લખ્યા હતા.

eng_Latn - guj_Gujr
